In [1]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [5]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Assuming 'data' and 'target' are 1D arrays
data = np.array(data).reshape(-1, 1)
target = np.array(target).reshape(-1, 1)

data_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = data_scaler.fit_transform(data)

target_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_target = target_scaler.fit_transform(target)


In [6]:
import joblib

joblib.dump(data_scaler,'data_scaler.sav')
joblib.dump(target_scaler,'target_scaler.sav')

['target_scaler.sav']

In [7]:
scaled_data=scaled_data.reshape(scaled_data.shape[0],scaled_data.shape[1],1)
scaled_target=scaled_target.reshape(scaled_target.shape[0],scaled_target.shape[1],1)

In [8]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(scaled_data,scaled_target,test_size=0.2)

In [9]:
from keras.layers import Input,Dense,Conv1D,MaxPool1D,UpSampling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint


input_layer=Input(shape=scaled_data.shape[1:])  

model=Conv1D(32,(3),activation='relu',padding='same')(input_layer)
model=MaxPool1D((2),padding='same')(model)
model=Conv1D(32,(3),activation='relu',padding='same')(model)
encoder=MaxPool1D((2),padding='same')(model)

model=Conv1D(32,(3),activation='relu',padding='same')(encoder)
model=UpSampling1D((2))(model) 
model=Conv1D(32,(3),activation='relu',padding='same')(model)
model=UpSampling1D((2))(model) 

decoder=Conv1D(1,(3),activation='relu',padding='same')(model)

autoencoder=Model(input_layer,decoder)

autoencoder.compile(optimizer='adam',loss='mse')

In [12]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('models/model-{epoch:03d}.keras', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
autoencoder.fit(train_data, train_target, epochs=10, validation_data=(test_data, test_target), callbacks=[checkpoint])


Epoch 1/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.0057 - val_loss: 0.0057
Epoch 2/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 48s 6ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 3/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 54s 2ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 4/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 5/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 6/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 86s 10ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 7/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 101s 5ms/step - loss: 0.0057 - val_loss: 0.0057
Epoch 8/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 110s 8ms/step - loss: 0.0056 - val_loss: 0.0057
Epoch 9/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 70s 7ms/step - loss: 0.0057 - val_loss: 0.0057
Epoch 10/10
8545/8545 ━━━━━━━━━━━━━━━━━━━━ 75s 9ms/step - loss: 0.0057 - val_loss: 0.0057


In [13]:
np.save('test_data',test_data)
np.save('test_target',test_target)